In [1]:
using JuMP, Gurobi

In [2]:
using MacroTools

In [3]:
expr = :(
    if x <= 0
            -x
    else
        zero(x)
    end
)

:(if x <= 0 # In[3], line 3:
        -x
    else  # In[3], line 5:
        zero(x)
    end)

In [170]:
module Ifs

using JuMP
using JuMP: AbstractJuMPScalar
using MacroTools: @capture

getmodel(x::JuMP.Variable) = x.m
getmodel(x::JuMP.GenericAffExpr) = x.vars[1].m

struct Condition{Op, T1, T2}
    op::Op
    lhs::T1
    rhs::T2
end

Base.show(io::IO, c::Condition{Op}) where {Op} = print(io, c.lhs, " ", c.op, " ", c.rhs)

complement(c::Condition{typeof(<=)}) = Condition(c.op, -c.lhs, -c.rhs)

struct Implication{C1 <: Condition, C2 <: Condition}
    lhs::C1
    rhs::C2
end

Base.show(io::IO, cv::Implication) = print(io, cv.rhs, " if ", cv.lhs)

function _condition(ex::Expr)
    if @capture(ex, op1_(l1_, r1_) => op2_(l2_, r2_))
        quote
            Implication(
                Condition($(esc(op)), $(esc(l1)), $(esc(r1))),
                Condition($(esc(op)), $(esc(l2)), $(esc(r2))))
                
        end
    elseif @capture(ex, op_(lhs_, rhs_))
        quote
            Condition($(esc(op)), $(esc(lhs)), $(esc(rhs)))
        end
    else
        error("Could not parse: $ex")
    end
end

macro condition(ex)
    _condition(ex)
end

upper_bound(m::Model, g::AbstractJuMPScalar) = 10

function implies(m::Model, z::AbstractJuMPScalar, c::Condition{typeof(<=)})
    g = c.lhs - c.rhs
    M = upper_bound(m, g)
    @constraint m c.lhs <= c.rhs + M * (1 - z)
end 

function implies(m::Model, z::AbstractJuMPScalar, c::Condition{typeof(==)})
    g = c.lhs - c.rhs
    M_u = upper_bound(m, g)
    @constraint(m, c.lhs - c.rhs <= M_u * (1 - z))
    M_l = -upper_bound(m, -g)
    @constraint(m, c.lhs - c.rhs >= M_l * (1 - z))
end 

function exclusive(m::Model, i1::Implication, i2::Implication)
    z = @variable(m, category=:Bin, basename="z")
    implies(m, z, i1.lhs)
    implies(m, z, i1.rhs)
    implies(m, 1 - z, i2.lhs)
    implies(m, 1 - z, i2.rhs)
end

function implies(m::Model, imp::Implication)
    z = @variable(m, category=:Bin, basename="z")
    implies(m, z, imp.lhs)
    implies(m, 1 - z, complement(imp.lhs))
    implies(m, z, imp.rhs)
end

macro switch(ex)
    body, cond_expr = if @capture(ex, if c1_
                v1_
            end)
        cond_expr = _condition(c1)
        quote
            cond = $(cond_expr)
            m = getmodel(cond.lhs)
            y = $(Expr(:macrocall, Symbol("@variable"), :m, Expr(:(=), esc(:basename), "y")))
            implies(m, Implication(cond, Condition(==, y, $(esc(v1)))))
            y
        end, cond_expr
    elseif @capture(ex, if c1_; v1_; else v2_; end)
        cond_expr = _condition(c1)
        quote
            cond = $(_condition(c1))
            comp = complement(cond)
            m = getmodel(cond.lhs)
            y = $(Expr(:macrocall, Symbol("@variable"), :m, Expr(:(=), esc(:basename), "y")))
            exclusive(m, 
                Implication(cond, Condition(==, y, $(esc(v1)))),
                Implication(comp, Condition(==, y, $(esc(v2)))))
            y
        end, cond_expr
    else
        error("Could not parse: $ex")
    end
    quote
        if isa($(cond_expr).lhs, AbstractJuMPScalar)
            $body
        else
            $(esc(ex))
        end
    end
end
            

end

Ifs

In [171]:
macroexpand(:(
    @Ifs.switch if x <= 0
    5
end
))

quote  # In[170], line 107:
    if begin  # In[170], line 37:
                    (Ifs.Condition)(<=, x, 0)
                end.lhs isa Ifs.AbstractJuMPScalar # In[170], line 108:
        begin  # In[170], line 85:
            #1429#cond = begin  # In[170], line 37:
                    (Ifs.Condition)(<=, x, 0)
                end # In[170], line 86:
            #1430#m = (Ifs.getmodel)(#1429#cond.lhs) # In[170], line 87:
            #1431#y = begin  # /Users/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/JuMP/src/macros.jl, line 251:
                    (JuMP.validmodel)(#1430#m, Symbol("#1430#m")) # /Users/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/JuMP/src/macros.jl, line 252:
                    #1432###1910 = (JuMP.constructvariable!)(#1430#m, JuMP._error, -Inf, Inf, :Default, (JuMP.string)("y"), NaN)
                end # In[170], line 88:
            (Ifs.implies)(#1430#m, (Ifs.Implication)(#1429#cond, (Ifs.Condition)(Ifs.==, #1431#y, begin  # In[171], 

In [172]:
x = 5
@Ifs.switch if x <= 0
    5
else
    6
end

6

In [175]:
function update(x)
    Ifs.@switch if x <= 0
        1
    else
        -1
    end
end

update (generic function with 1 method)

In [193]:
update(0.5)

-1

In [194]:
m = Model()
@variable m x
update(x)

y

In [195]:
m

Feasibility problem with:
 * 6 linear constraints
 * 3 variables: 1 binary
Solver is default solver

In [191]:
x = 0.5
ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end
ys

4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [192]:
m = Model(solver=GurobiSolver())
@variable m x
@constraint m x == 0.5

ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end

solve(m)
getvalue.(ys)

Optimize a model with 19 rows, 7 columns and 37 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+01]
Presolve removed 19 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [182]:
@code_warntype update(0.5)

Variables:
  #self#::#update
  x::Float64
  cond::Any
  comp::Any
  m::Any
  y::Any
  ##1915::Any
  fy::Float64

Body:
  begin 
      # meta: location In[170] # line 37:
      SSAValue(12) = x::Float64
      goto 6 # line 101:
      6:  # line 110:
      $(Expr(:inbounds, false))
      # meta: location float.jl <= 495
      fy::Float64 = (Base.sitofp)(Float64, 0)::Float64
      # meta: pop location
      $(Expr(:inbounds, :pop))
      unless (Base.or_int)((Base.lt_float)(x::Float64, fy::Float64)::Bool, (Base.and_int)((Base.and_int)((Base.eq_float)(x::Float64, fy::Float64)::Bool, (Base.lt_float)(fy::Float64, 9.223372036854776e18)::Bool)::Bool, (Base.sle_int)((Base.fptosi)(Int64, fy::Float64)::Int64, 0)::Bool)::Bool)::Bool goto 15
      return 1
      15: 
      # meta: pop location
      return -1
  end::Int64


In [164]:
m = Model(solver=GurobiSolver())
@variable m x

y = @Ifs.switch if x <= 0
    5
else
    3
end
@constraint m x == 1


@objective m Min x^2 + y^2
solve(m)
getvalue(y)

Optimize a model with 7 rows, 3 columns and 13 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 7 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 10 
Pool objective bound 10

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


3.0000000000000053

In [162]:
m = Model(solver=GurobiSolver())
@variable m x

y = @Ifs.switch if x <= 0
    5
end
@constraint m x == -1


@objective m Min x^2 + y^2
solve(m)
getvalue(y)

Optimize a model with 5 rows, 3 columns and 9 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 26 
Pool objective bound 26

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+01, best bound 2.600000000000e+01, gap 0.0000%


5.000000000000006

In [76]:
m = Model(solver=GurobiSolver())
@variable m x
@variable m y
@objective m Min x^2 + y^2

Ifs.implies(m, @Ifs.condition(x <= 0 => y == 5))
@constraint m x == 1
solve(m)
getvalue(y)

Optimize a model with 5 rows, 3 columns and 9 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1 
Pool objective bound 1

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


-0.0

In [55]:
c = @Ifs.condition(x == 2 => 3)

3 if x == 2

In [ ]:
m = Model(solver=GurobiSolver())
@variable m x
y = Ifs.exclusive(m, @Ifs.condition(x <= 0))

In [41]:
m = Model(solver=GurobiSolver())
@variable m x



@variable m z Bin
Ifs.implies(m, z, @Ifs.condition x == 2)
@constraint m z == 0
solve(m)
getvalue(x)

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Variable types: 1 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 1e+01]
Found heuristic solution: objective 0

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


0.0

In [39]:
m

Feasibility problem with:
 * 2 linear constraints
 * 2 variables: 1 binary
Solver is Gurobi

In [28]:
m = Model()
@variable m x
c = Ifs.@condition x == 5

x == 5

In [16]:
m = Model(solver=GurobiSolver())
@variable m x

options = (
    ConditionalValue(
        Condition{:(<=)}(x, 0),
        -x),
    ConditionalValue(
        Condition{:(<=)}(-x, -0),
        zero(x))
        )

(-x if x <= 0, 0 if -x <= 0)

In [33]:
function dynamics(x)
    f = if isa(x, JuMP.AbstractJuMPScalar)
        M = 10
        model = getmodel(x)
        z = @variable(model, category=:Bin, basename="z")
        @constraint(model, x <= 0 + M * (1 - z))
        @constraint(model, x >= 0 - M * z)
        y = @variable(model, basename="y")
        @constraint(model, y <= -x + M * (1 - z))
        @constraint(model, y >= -x - M * (1 - z))
        @constraint(model, y <= zero(x) + M * z)
        @constraint(model, y >= zero(x) - M * z)
        y
    else
        if x <= 0
            -x
        else
            zero(x)
        end
    end
    y = x + 0.1 * f
end

dynamics (generic function with 1 method)

In [36]:
m = Model(solver=GurobiSolver())
@variable m x
@objective m Min y^2
@constraint m x == -1
y = dynamics(x)
ys = [y]
for i in 1:5
    push!(ys, dynamics(ys[end]))
end

In [37]:
solve(m)
getvalue(x), getvalue.(ys)

LoadError: [91mUndefVarError: VariableNotOwnedException not defined[39m

In [4]:
x <= 1

LoadError: [91mMethodError: no method matching isless(::Int64, ::JuMP.Variable)[0m
Closest candidates are:
  isless(::Real, [91m::AbstractFloat[39m) at operators.jl:97
  isless(::Real, [91m::ForwardDiff.Dual[39m) at /Users/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/ForwardDiff/src/dual.jl:161
  isless(::Real, [91m::Real[39m) at operators.jl:266[39m

In [5]:
expr = :(
if x <= 1
    k * x
else
    0
end
)

:(if x <= 1 # In[5], line 3:
        k * x
    else  # In[5], line 5:
        0
    end)

In [8]:
supertype(JuMP.Variable)

JuMP.AbstractJuMPScalar

In [9]:
supertype(JuMP.AffExpr)

JuMP.AbstractJuMPScalar

In [11]:
if isa(x, JuMP.AbstractJuMPScalar)
    M = 10
    z = @variable(x.m, category=:Bin, basename="z")
    @constraint(x.m, x <= 1 + M * (1 - z))
    y = @variable(x.m, basename="y")
    @constraint(x.m, y <= 1 + M * (1 - z))
    @constraint(x.m, y >= 1 - M * (1 - z))
    @constraint(x.m, y <= 2 + M * z)
    @constraint(x.m, y >= 2 - M * z)
    y
end

y

In [7]:
dump(:(x <= 1 ? 1 : 2))

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol <=
        2: Symbol x
        3: Int64 1
      typ: Any
    2: Int64 1
    3: Int64 2
  typ: Any


In [6]:
dump(expr)

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol <=
        2: Symbol x
        3: Int64 1
      typ: Any
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol line
          args: Array{Any}((2,))
            1: Int64 3
            2: Symbol In[5]
          typ: Any
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol k
            3: Symbol x
          typ: Any
      typ: Any
    3: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol line
          args: Array{Any}((2,))
            1: Int64 5
            2: Symbol In[5]
          typ: Any
        2: Int64 0
      typ: Any
  typ: Any
